# Day 3 - Rucksack Reorganization

>One Elf has the important job of loading all of the rucksacks with supplies for the jungle journey. Unfortunately, that Elf didn't quite follow the packing instructions, and so a few items now need to be rearranged.
>
>Each rucksack has two large compartments. All items of a given type are meant to go into exactly one of the two compartments. The Elf that did the packing failed to follow this rule for exactly one item type per rucksack.
>
>The Elves have made a list of all of the items currently in each rucksack (your puzzle input), but they need your help finding the errors. Every item type is identified by a single lowercase or uppercase letter (that is, a and A refer to different types of items).
>
>The list of items for each rucksack is given as characters all on a single line. A given rucksack always has the same number of items in each of its two compartments, so the first half of the characters represent items in the first compartment, while the second half of the characters represent items in the second compartment.
>
>For example, suppose you have the following list of contents from six rucksacks:
>```
>vJrwpWtwJgWrhcsFMMfFFhFp
>jqHRNqRjqzjGDLGLrsFMfFZSrLrFZsSL
>PmmdzqPrVvPwwTWBwg
>wMqvLMZHhHMvwLHjbvcjnnSBnvTQFn
>ttgJtRGJQctTZtZT
>CrZsJsPPZsGzwwsLwLmpwMDw
>```
>- The first rucksack contains the items vJrwpWtwJgWrhcsFMMfFFhFp, which means its first compartment contains the items vJrwpWtwJgWr, while the second compartment contains the items hcsFMMfFFhFp. The only item type that appears in both compartments is lowercase p.
>- The second rucksack's compartments contain jqHRNqRjqzjGDLGL and rsFMfFZSrLrFZsSL. The only item type that appears in both compartments is uppercase L.
>- The third rucksack's compartments contain PmmdzqPrV and vPwwTWBwg; the only common item type is uppercase P.
>- The fourth rucksack's compartments only share item type v.
>- The fifth rucksack's compartments only share item type t.
>- The sixth rucksack's compartments only share item type s.
>
>To help prioritize item rearrangement, every item type can be converted to a priority:
>
>- Lowercase item types a through z have priorities 1 through 26.
>- Uppercase item types A through Z have priorities 27 through 52.
>
>In the above example, the priority of the item type that appears in both compartments of each rucksack is 16 (p), 38 (L), 42 (P), 22 (v), 20 (t), and 19 (s); the sum of these is 157.

## Read input

Today, the data input is straight-forward: since each line is a string input, there's no need for any extra manipulation.

In [50]:
from utils import read_input

rucksacks = read_input(3)
example = read_input(3, example=True)

## Part 1

For the first part, we need to split each input line into two halves and find common item between those two parts.

A great way to find common items in any collections is to convert them to [sets](https://docs.python.org/3/tutorial/datastructures.html#sets). A set is a data structure which keeps each item only once. It's very efficient in certain operations like checking if a value is part of a set and for taking a union (all values in two sets combined) or an intersection (shared values in two sets). The latter is exactly what we need today.

To get an intersection of two sets in Python, there are two options: you can either do `a.intersect(b)` or `a & b`.

We'll also need to be able to calculate the priority value for these items. The values are 1-26 for a-z and 27-52 for A-Z. That also happens to be their index + 1 in the `string.ascii_letters` string.

In [52]:
import string

def calculate_priority(characters):
    """
    > calculate_priority({'a'})
    1
    > calculate_priority({'a', 'A'})
    28
    > calculate_priority({})
    0
    """
    return sum(string.ascii_letters.index(char) + 1 for char in characters)

> **Find the item type that appears in both compartments of each rucksack. What is the sum of the priorities of those item types?**

In [57]:
def calculate_total_priority(rucksacks):
    priority = 0
    for rucksack in rucksacks:
        mid_point = len(rucksack) // 2
        first, second = set(rucksack[:mid_point]), set(rucksack[mid_point:])
        common = first & second
        priority += calculate_priority(common)

    return priority

priority = calculate_total_priority(rucksacks)
print(f'Part 1: {priority}')
assert priority == 7826

Part 1: 7826


## Part 2

> As you finish identifying the misplaced items, the Elves come to you with another issue.
>
>For safety, the Elves are divided into groups of three. Every Elf carries a badge that identifies their group. For efficiency, within each group of three Elves, the badge is the only item type carried by all three Elves. That is, if a group's badge is item type B, then all three Elves will have item type B somewhere in their rucksack, and at most two of the Elves will be carrying any other item type.
>
>The problem is that someone forgot to put this year's updated authenticity sticker on the badges. All of the badges need to be pulled out of the rucksacks so the new authenticity stickers can be attached.
>
>Additionally, nobody wrote down which item type corresponds to each group's badges. The only way to tell which item type is the right one is by finding the one item type that is common between all three Elves in each group.
>
>Every set of three lines in your list corresponds to a single group, but each group can have a different badge item type. So, in the above example, the first group's rucksacks are the first three lines:
>```
>vJrwpWtwJgWrhcsFMMfFFhFp
>jqHRNqRjqzjGDLGLrsFMfFZSrLrFZsSL
>PmmdzqPrVvPwwTWBwg
>```
>
>And the second group's rucksacks are the next three lines:
>
>```
>wMqvLMZHhHMvwLHjbvcjnnSBnvTQFn
>ttgJtRGJQctTZtZT
>CrZsJsPPZsGzwwsLwLmpwMDw
>```
>
>In the first group, the only item type that appears in all three rucksacks is lowercase r; this must be their badges. In the second group, their badge item type must be Z.
>
>Priorities for these items must still be found to organize the sticker attachment efforts: here, they are 18 (r) for the first group and 52 (Z) for the second group. The sum of these is 70.

I spent way too much time on building the wrong thing today. I somehow missed that the groups were already in the input (each 3 consecutive lines == group) and I built functions that found all the possible groups in the input, only to find too many of them.

Once I figured what the puzzle wanted, the code become quite a lot cleaner.

For each group of 3 racksacks, we find the common unique item and get its priority and then add them all together.

In [58]:
def unique_among_group(group):
    return set(group[0]) & set(group[1]) & set(group[2])
    
priority = 0
for idx in range(0, len(rucksacks), 3):
    badge = unique_among_group(rucksacks[idx:idx+3])
    priority += calculate_priority(badge)
    
print(f'Part 2: {priority}')
assert priority == 2577

Part 2: 2577


## Doctest runner

I'll keep the doctest runners at the bottom of the file for future. I originally had way more doctests in this solution until I learned that I had done wrong (and way too much) work in the second part.

In [60]:
import doctest

doctest.testmod()

TestResults(failed=0, attempted=4)